In [64]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import Markdown, Image
# for presentations:
#display(HTML("<style>.container { width:100% !important; }</style>"))


def lw_comment(n:int):
    """Create a line width indicator comment line to, e.g.
    place as 1st line in a md doc.
    """
    x = n - 5
    line = "<!---" +  ".".join("" for i in range(x)) + f"|:{n} --->"

    return line

# Flowcharts for MCCE_Benchmarking:

Note: GitHub markdown apparently cannot render the mermaid graphs despite saying it does.


## Purpose:

To allow benchmarking viz the curated, experimental pKa database (v1). **DONE**.  
To allow benchmarking of mcce runs. **DONE**  

---
---

# Installation.md - editing

In [66]:
Markdown(filename="../Installation.md")

<!---..................................................................................................................|:120 --->
# Installing MCCE_Benchmarking & Testing:
  * Notation remarks:
    a. ">" denotes the command line prompt (whatever it may be in your profile);
    b. ">>" denotes the output of a command.
    c. On a command line, "<x>" is an example of an argument value.

## Start
  ### 0. Preferably: `>cd $HOME`

  ### 1. Install miniconda, if needed  
    1.2 (Optional) Configure .condarc for installation channels & packages included with all new env, e.g.:
    <details>
        <summary>[conda config]</summary>

        ```sh

        # user_rc_path
        channel_priority: strict
        channels:
          - conda-forge
          - bioconda
          - newbooks

        auto_update_conda: true
        report_errors: true

        # disallow install of specific packages
        disallow:
          - anaconda
        # Add pip, wheel, and setuptools as deps of Python
        add_pip_as_python_dependency: true

        create_default_packages:
          - numpy
          - ipython
          - ipykernel
          - python-dotenv
          - matplotlib
          - pandas
          - pytest
          - black
          - pylint
          - flake8
        ```
</details>

  ### 2. Create a dedicated environment for all things mcce:
    - e.g. 'mce'
     ```sh

     >conda create -n mce python=3.11   # the version part (=3.11) is optional, but not python

     ```
   - Activate the env:
     ```sh

     >conda activate mce  # => new prompt: (mce) >

     ```
   - Install mcce
     * For Gunner Lab members:
       [UPDATE, 03-01-2024:
       This is a MUST: the packaged 'Stable-MCCE' is missing a fortran library;
       See Issue 282, https://github.com/GunnerLab/Stable-MCCE/issues/282
       ]
       On the server, mcce is already installed, but it must be in your path variable; 
       Add the following line in your .bashrc, save & close the file, then source it:
       ```sh
       export PATH="/home/mcce/Stable-MCCE/bin:$PATH"
       ```
       ```sh

       (mce) > . ~/.bashrc   # source or 'dot' the file

       (mce) >which mcce     # should return the added path

       ```

     * For all others, install the package if Issue 282 is resolved:
       ```sh

       (mce) >conda install -c newbooks mcce

       ```

  ### 3. Install mcce_benchmark (right now from GitHub, not yet published on pypa):
  `(mce) >pip install git+https://github.com/GunnerLab/MCCE_Benchmarking.git#egg=mcce_benchmark`

  #### 3.1 Check that the installation created the 4 cli commands:
  (The commands will NOT be available outside your environment.)

  ```sh
  (mce) >which bench_setup
  >>~/miniconda3/envs/mce/bin/bench_setup

  (mce) >which bench_launchjob
  >>~/miniconda3/envs/mce/bin/bench_launchjob

  (mce) >which bench_analyze
  >>~/miniconda3/envs/mce/bin/bench_analyze

  (mce) >which bench_compare
  >>~/miniconda3/envs/mce/bin/bench_compare

  ```

  #### 3.2 Check the online help, e.g.:
  ```sh

  (mce) >bench_setup pkdb_pdbs --help

  (mce) >bench_setup user_pdbs --help

  ```
---

# Command line options
Here is a short survey of the available options.

## 1. To create a set using the pdbs from pKaDBv1, use the setup command & sub-command `bench_setup pkdb_pdbs`.
Note: `bench_setup` creates the run script with the command line options for the mcce steps; if none are given, the default run script is used.
  * Minimal input `-bench_dir`:
    ```sh
    
    (mce) >bench_setup pkdb_pdbs -bench_dir <some/dir>  # -bench_dir is a required arg
    
    ```
    
    You can limit the number of curated proteins (folder) to setup using `-n_pdbs`.
    Give a low number for testing (max is 120, default):
    ```sh
    
    (mce) >bench_setup pkdb_pdbs -bench_dir <some/dir> -n_pdbs <2>
    
    ```

  * `-job_name` option
   The default job name is 'default_run'; you can change it using `-job_name`.
    - If you use `-job_name` without amending any of the steps args, the default_run.sh file will be soft-linked as <job name>;
    - If you pass one or more non-default mcce steps parameters, then you MUST have a job name (& not 'default_run'):
    ```sh

    (mce) >bench_setup pkdb_pdbs -bench_dir <some/dir> -n_pdbs <2> -d 8 -job_name <foo_e8>
    
    ```

  * `-sentinel_file` option:
  The default sentinel_file is **pK.out** (means the run completed Step 4). It is part of script setup
  to ensure it is deleted prior to launching. You need to include it if you are not running all 4 steps. Example running only steps 1 & 2:
  ```sh

  (mce) >bench_setup [pkdb_pdbs or user_pdbs] -bench_dir <some/dir> --s3_norun  --s4_norun -job_name <up_to_s2> -sentinel_file step2_out.pdb

  ```

  * `--launch` option (flag):
  Launching a job with `bench_setup` is possible with the `--launch` flag:
  ```sh

  (mce) >bench_setup [pkdb_pdbs or user_pdbs] -bench_dir <some/dir> -d 8 -job_name <foo_e8> --launch

  ```

  In this case, you CANNOT review or amend the run script. You would not launch the job this way if, for instance,  
  you want to add Step 6.
  **TODO**: Step 6 will be part of the script once a hydrogen-bond network comparison procedure is finalized.

## 2. To create a set using a user-provided list of pdbs, use the setup command & sub-command `bench_setup user_pdbs`.

  * Minimal input `-bench_dir`, `-pdbs_list`:
  ```sh

  (mce) >bench_setup user_pdbs -bench_dir <some/dir>  -pdbs_list ./path/to/dir or ./path/to/file/of/pdbs filepaths

  ```

  * `-job_name` option: same usage as with `pkdb_pdbs` sub-command

  * `-sentinel_file` option: same usage as with `pkdb_pdbs` sub-command

  * `--launch` option (flag): same usage as with `pkdb_pdbs` sub-command

## 3. To launch batches of jobs, use `bench_setup launch`
   * Use the automated scheduling to process all the pdbs in batches of size n_batch (default 10):
   ```
   (mce) >bench_setup launch -bench_dir <some/dir> -job_name <foo> -n_batch <4> [-sentinel_file x]  # needed if used in setup

   ```

## 4. Analyze (one set):

The command line interface for analyzing a job setup via `bench_analyze [pkdb_pdbs or user_pdbs]`; requires 1 argument: `-bench_dir`.
The first step of the analysis checks that all the runs are completed. Incidentally, it's an easy way to check how far along the processing is.

  * Output files & sub-commands:
    - user_pdbs: Collated pK.out and sum_crg.out, res and confs count files & related figures
    - pkdb_pdbs: Analysis additionally includes res and confs stats viz experimental pKa values and related figures.


## 5. Compare two sets of runs, i.e. A/B testing (convention: B is reference):

To compare two (completed) sets of runs, use `bench_compare`:
  * Options: -di1, -dir2 -o (output folder), and two flags:
    * `--user_db_pdbs`      # absence means 'pkdb_pdbs'
    * `--dir2_is_refset`    # if used, --user_pdbs must NOT be present, e.g.:
    ```
    (mce) >bench_compare -dir1 <d1> dir2 parse.e4 --dir2_is_refset -o <d1/comp>

    ```


## 6. Give feedback in repo:
Please, open an issue with any problem installing or using the app, along with enhancement requests on the [MCCE_Benchmarking issues](https://github.com/GunnerLab/MCCE_Benchmarking/issues) page. Thanks.

<!---..................................................................................................................|:120 --->

# USER ROADMAP - 03-14-2024

## Four choices in MCCE_Benchmarking, `mcce_benchmark` (a.k.a. "the app"):
### 1. Create a new benchmarking set (with a "launch right away" option)
### 2. Launch the automated runs for the set
### 3. Analyze one set of runs
### 4. Compare two sets of runs

#### All these correspond to command line "entry points", EPs. The flowcharts below provide an overview. To learn about the options available for each EP/subcommands [see Details](#Details)
---
## Flowchart for the current entry points (EPs):

### The entry point `bench_setup +[user_pdbs or pkdb_pdbs sub-command]` handles the data & script setup:
```mermaid
%%{init: {"flowchart": {"htmlLabels": false}} }%%
flowchart LR
    one(("`__1__`")) ~~~ create["`__Create a new benchmarking set__
    **CMD :: bench_setup**`"] --> pdbs{Use pdbs in pKaDB?}
    pdbs -->|N| l1{launch right away?}
    l1 -->|Y| mcbY("`__bench_setup user_pdbs__ <args> --launch`")
    l1 -->|N| mcbN("`__bench_setup user_pdbs__ <args>`")
    pdbs -->|Y| phDB{"`__pH?__`"}
    phDB -->|Y| l2{launch right away?}
    l2 -->|Y| explY("`__bench_setup pkdb_pdbs__ <args> --launch`")
    l2 -->|N| explN("`__bench_setup pkdb_pdbs__ <args>`")
    phDB -->|N| nogo["`__Invalid choice:__
    pKaDB pdbs have no cofactors`"]
```

### The entry point `bench_setup launch` is used if `--launch` was not used during data & script setup:
```mermaid
%%{init: {"flowchart": {"htmlLabels": false}} }%%
flowchart LR
    two(("`__2__`")) ~~~ launch["`__Launch automated processing of the set__`"] --> do("`__bench_setup launch__ <args>`")
```

### The entry point `bench_analyze` does an "intra set" analysis

```mermaid
%%{init: {"flowchart": {"htmlLabels": false}} }%%

flowchart LR
    thre(("`__3__`")) ~~~ anlyze["`__Analyze 1 set__`"] --> orig{Set created
     by app?}
    orig -->|Y| pdbs{Set pdbs
    from pKaDB?}
    pdbs -->|N| mcb("`__bench_analyze user_pdbs__`")
    pdbs -->|Y| phDB{"`__pH?__`"}
    phDB -->|Y| expl("`__bench_analyze pkdb_pdbs__`")
    phDB -->|N| nogo1["`__Invalid choice__:
    pKaDB pdbs have no cofactors`"]
    orig -->|N| nogo2["`__Not possible__
    Expected structure:
    .'bench_dir'
    |--RUNS/
      |--PDB1/
         |--run.prm.record
      [...]
      --book.txt`"]
```

### The entry point `bench_compare` does an "inter sets" analysis

```mermaid
%%{init: {"flowchart": {"htmlLabels": false}} }%%

flowchart LR
    four(("`__4__`")) ~~~ comp["`__Compare 2 sets__`"] --> orig{Sets created
     by app?}
    orig -->|Y| pdbs{Sets pdbs
    from pKaDB?}
    pdbs -->|N| user("`__bench_compare -dir1 ./A -dir2 ./B --user_pdbs__`")
    pdbs -->|Y| phDB{"`__pH?__`"}
    phDB -->|Y| ref{Is dir2 a
    reference set?
    i.e. parse.e4}
    ref -->|N| pkdbN("`__bench_compare -dir1 ./A -dir2 ./B__`")
    ref -->|Y| pkdbY("`__bench_compare -dir1 ./A -dir2 ./B --dir2_is_refset__`")
    phDB -->|N| nogo1["`__Invalid choice__:
    pKaDB pdbs have no cofactors`"]
    orig -->|N| nogo2["`__Not possible__
    Expected structure:
    .'bench_dir'
    |--RUNS/
      |--PDB1/
         |--run.prm.record
      [...]
      --book.txt`"]
    four ~~~ note["`In __A/B__ testing, B is the reference set, i.e:
    __dir1::A; dir2::B__
    A vs. B
    A - B
    New vs. existing
    Y vs. X`"]
```
  * Note: The flag `--dir2_is_refset` indicates that 'dir2' is the _name_ of a packaged reference dataset, currently 'parse.e4' (pH titrations using the pKaDBv1 pdbs.); without it, dir2 is a path.
---
---

# Details


# 03-08-2024
---
---
# USERS ROADMAP

## Three main choices in MCCE_Benchmarking
### 1. Create a new benchmarking set
### 2. Analyze one set of runs
### 3. Compare two sets

---

```mermaid
%%{init: {"flowchart": {"htmlLabels": false}} }%%
flowchart LR
    create["`__1. Create a new benchmarking set__`"] --> pdbs{Use pdbs in pKaDB?}
    pdbs -->|N| mcb("`__bench_mcce_runs setup_job__`")
    pdbs -->|Y| phDB{"`__pH?__`"}
    phDB -->|Y| expl("`__bench_expl_pkas setup_job__`")
    phDB -->|N| nogo["`__Invalid choice:__
    pKaDB pdbs have no cofactors`"]
```



```mermaid
%%{init: {"flowchart": {"htmlLabels": false}} }%%

flowchart LR
    anlyze["`__2. Analyze one set__`"] --> orig{Set created
     by app?}
    orig -->|Y| pdbs{Set pdbs
    from pKaDB?}
    pdbs -->|N| mcb("`Use: __bench_analyze mcce_runs__`")
    pdbs -->|Y| phDB{"`__pH?__`"}
    phDB -->|Y| expl("`Use: __bench_analyze expl_pkas__`")
    phDB -->|N| nogo1["`__Invalid choice__:
    pKaDB pdbs have no cofactors`"]
    orig -->|N| nogo2["`__Not possible__
    Expected structure:
    .'benchmarks_dir'
    |--mcce_runs/
      |--PDB1/
         |--run.prm.record
      [...]
      --book.txt`"]
```




```mermaid
%%{init: {"flowchart": {"htmlLabels": false}} }%%

flowchart LR
    anlyze["`__3. Compare two sets: TODO__
    Main args: set1_dir, set2_dir`"] --> orig{Sets created
     by app?}
    orig -->|Y| pdbs{Sets pdbs
    from pKaDB?}
    pdbs -->|Y| expl("`__bench_compare pkadb_runs set1_dir set2_dir...__`")
    pdbs -->|N| mcb1("`__bench_compare mcce_runs set1_dir set2_dir...__`")
    orig -->|N| single{"`Sets are single
    mcce runs on the
    same protein?`"}
    single -->|Y| mcb2("`__bench_compare mcce_runs set1_dir set2_dir... --singles__`")
    single -->|N| nogo["`__Not possible__
    Expected structure:
    .'benchmarks_dir'
    |--mcce_runs/
      |--PDB1/
         |--run.prm.record
      [...]
      --book.txt`"]
```


# 03-05-2024
---
# Update post 3/4/24 group meeting on benchmarking
  * Variable names change:
    - n_active -> n_batch
  * Added missing scipy requirement (needed for mcce, not mcce_benchmarking per se)


## Completion status:
 * Entry point 'bench_expl_pkas' with sub-commands 'setup_job' and 'launch_job': DONE
 * Entry point 'bench_mcce_runs': 30%

---
## Flowchart for the current entry points (EPs) of `mcce_benchmark`:

```mermaid
%%{init: {"flowchart": {"htmlLabels": false}} }%%
flowchart TD
    exp("`EP:  _**bench_expl_pkas**_
    <br>
    Status: **DONE**
    <br>
    sub-cmd 1: _setup_job_
    sub-cmd 2: _launch_job_`"
    ) -- 1--> sb1[setup_job]
    exp -- 2 --> sb2[launch_job]
    sb1 -- Input: 'benchmarks_dir' --> dir1[\-benchmarks_dir exists?/]
    dir1 -->|N| ng1[mkdir 'benchmarks_dir']
    dir1 -->|Y| dosb1[Actions performed:
    1. Data setup
    2. Automated  cron scheduling
    ]
    ng1 --> dosb1
    sb2 --> dosb2[Create scheduling via crontab]
    dosb2 -- Monitor % completion --> cmd1[>grep -i 'completed' benchmarks_dir/cron_job_name.log]

    batch("`EP:  _**bench_launchjob**_
    (used in crontab)
    <br>
    Status: **DONE**
    <br>
    Submit ONE batch of size -n_batch
    using sh script`"
    ) -- Pre-requisites:
    A setup as in `bench_expl_pkas setup_job`:
     - Same folder structure
     - 'default_run.sh' or 'job_name.sh' script
        if job_name was used. --> wait(Wait x min)
     wait -->|Monitor % completion| cmd2[>grep -i 'completed' ../benchmark.log]
     cmd2 -->|Repeat until all jobs are completed| batch

    anz("`EP: _**bench_analyze**_
    <br>
    Status: **50% done**
    <br>
    sub-cmd 1: expl_pkas
    sub-cmd 2: mcce_runs`"
    ) -->ok[\Are all the jobs completed?/]
    ok -->|N| nogo1(Check again later!)
    ok -->|Y|asb1[1. expl_pkas]
    ok -->|Y|asb2[2. mcce_runs TODO]
    asb1 -->|Pre-requisites:
    Same folder structure as in
    `bench_expl_pkas setup_job`| dir2[\-benchmarks_dir exists?/]
    dir2 -->|Y|doasb1[Create all_pkas.out
    Output report files
    and figures in
    benchmarks_dir/analysis]
    dir2 -->|N| npgp2(Oops! Typo? Wrong dir?)
    asb2 -->|Pre-requisites:
    completed runs| doasb2["`**TBD**:
    diff of pK.out, sum_crg.out`"]
```
```mermaid
%%{init: {"flowchart": {"htmlLabels": false}} }%%

flowchart LR
subgraph sg0 ["`**Note:**`"]
    o["Because the folder structure and and pdbs will be different:"]
    style o fill:#fff,stroke:#f66,stroke-width:1px

    subgraph sg1 ["expl_pKas: pH only, pkadb pdbs & setup"]
    direction LR
      a("`**bench_expl_pkas**`") ==> b["`_bench_analyze_ **expl_pkas**`"]
    end
    subgraph sg2 ["mcce_runs**: pH, Eh, different setup..."]
    direction LR
      c("`**bench_mcce_runs**`") ==>d["`_bench_analyze_ **mcce_runs**`"]
    end

end
```

---
---
# 2-14-2024
---
# Refactoring MCCE benchmark cli ([Issue 16](https://github.com/GunnerLab/MCCE_Benchmarking/issues/16))

## Purpose:
To allow benchmarking viz the curated, experimental pKa database (v1) or viz MCCE runs (two runs with completed step4).

## Completion status:
The refactoring is not complete yet as it entails creating two different tracks (or entry points), along with automated tests.
* 'experimental_pkas': 80%
* 'mcce_runs': 30%

## What the command line commands would be (w/o renaming main EP):
### Entry point 'experimental_pkas':
 1. Setup the pdbs folder and files for the user. NO GO: `benchmarks_dir` already exists
 ```
 # No input means `benchmarks_dir` = ./mcce_benchmarks:

 >mccebench experimental_pkas benchmark_setup


 # With input:

 >mccebench experimental_pkas benchmark_setup -benchmarks_dir <different name>


 ```
 2. Analysis (default for `pct_complete` not yet determined):
 ```
 >mccebench experimental_pkas analyze -pct_complete <float> [-benchmarks_dir: optional if default was used in `benchmark_setup`]
 ```

### Entry point 'mcce_runs':
NOTE: I think there is no need for the -eps parameter since we would want to compare, say e8 vs e4 (default); Updated the flowchart accordingly.  
NO GO: User selected -titr_type = 'eh' with -reference_dir = 'parse.e4': this reference set is for ph titrations.
```
 >mccebench mcce_runs -new_calc_dir <new>, -reference_dir <ref> -titr_type <ph or eh>


```

---
## Flowchart for the two entry points:

```mermaid
graph TD
    exp[Entry point: 'experimental_pkas'
    default script: dry prot, 4 steps w/mfe] -->|Input: 'benchmards_dir'| dir{dir exists?}
    dir -->|N| ng1[STOP:
    Rename existing dir or
    Change 'benchmarks_dir']
    dir -->|Y| sub1{Subcommand choice:
    1. 'benchmark_setup'
    2. 'analyze'}
    sub1 -->| 'benchmark_setup' |do1[Actions performed:
    1. Data setup
    2. Scheduling setup
    3. Launch]
    sub1 -->| 'analyze' | prob1[Problem:
    Are there enough completed runs?
    Implement with a 'percentage' user input?
    => Function needed as initial check
    to obtain the completed entries in
    the book file & launch the analysis
    if % is met.]
    prob1 --> runs1{Enough completed runs?}
    runs1 -->| Y | rpt1[Final outputs:
    * all_pkas file
    * Matched pKas file
    * Residue stats
    * Conformers throughput per step\n using runtimes & conformer counts
    * Plots
    * Anything else?]
    runs1 --> | N | msg1(Try '>experimental_pkas analyze' later)

    mc[Entry point: 'mcce_runs'] -->|Inputs:
    2 completed runs:
    'new_calc_dir', 'reference_dir';
    'titr_type'| ref{Which 'reference_dir'?}
    ref -->|ref dir is e.g. 'parse.e4' from pKaDB
    Applicable only to pH titrations| comp2[Use 'all_pkas.e4' file for comparison]
    comp2 --> mcpka[Analysis outputs:
    * Matched, then diffed pKa values
    * Plot new vs ref for all numeric fields in pK.out]
    ref -->|ref dir is another mcce output dir
    ASSUMED: runs of same prot| rptmc[Analysis outputs:
    * Diffed pK.out
    * Residue stats
    * Plot new vs ref for all numeric fields in pK.out
    * Anything else?]
    mcpka -.-> note[Problem:
    Analysis will depend on the
    contents of 'parse.e4' dir:
    full output, partial
    or just 'all_pkas.e4' file?]
    rptmc -.-> note
```